# Data Cleaning


Having data does not always allow you to produce some analytics right away. There is often a lot of pre processing to be done. 

This material is about **Cleaning**: making sure each cell has a value that could be used in your coming procedures. There are always some _impurities_ that do not allow the computer to recognize the data correctly, i.e. _commas_ instead of _periods_ and viceversa, the presence of unneeded _blanks_, irrelevant symbols (dollar, euro symbols), or non-standard symbols to represent missing values.

I will use two approaches. The first one is the smart use of regular expressions (**regex**), and the second one a **divide and conquer** strategy.

# REGEX VERSUS DIVIDE AND CONQUER 

Imagine that you request the age people in an online form. Sometimes you run into answers with issues like these:

- "It is:24"
- "It is: 44"
- "It is54"
- "64 it is"
- "I am twenty"
- "The 10th I turn 21"
- "I am 15 years old"
- "~20"

From the above examples, you are interested in the _age_, nothing else. The first two cases are _relatively_ easy to solve with divide and conquer, as you see a character that helps:  

In [1]:
case1="It is:24"
case2="It is: 34"
# try 1
case1.split(':')[1]

'24'

In [2]:
#try 2:
case2.split(':')[1]

' 34'

Split broke the string using ":" and produced a _list_.  The number will be the second element. However, in _case2_ you got an extra space. You need to think about a general rule, so maybe this is better:

In [3]:
case1.split(':')[1].strip()

'24'

In [4]:
case2.split(':')[1].strip()

'34'

Using _strip()_ gets rid of the spaces around the string.  Notice _strip()_ and _split()_ are functions in **base Python**. Pandas has its **own** functions. 

You can use the divide and conquer as long as every string you find follows the same pattern. Imagine those values make a column in a data frame:

In [5]:
import pandas as pd

ages=["It is:24","It is: 44","It is54",
      "64 it is","I am twenty","The 10th I turn 21",
      "I am 15 years old","~20"]

someData=pd.DataFrame({'age':ages})
someData

,age
0,It is:24
1,It is: 44
2,It is54
3,64 it is
4,I am twenty
5,The 10th I turn 21
6,I am 15 years old
7,~20


Now, let's use Pandas **own** strip and split:

In [6]:
someData.age.str.split(':')

0             [It is, 24]
1            [It is,  44]
2               [It is54]
3              [64 it is]
4           [I am twenty]
5    [The 10th I turn 21]
6     [I am 15 years old]
7                   [~20]
Name: age, dtype: object

Or even better:

In [7]:
someData.age.str.split(':',expand=True)

,0,1
0,It is,24
1,It is,44
2,It is54,None
3,64 it is,None
4,I am twenty,None
5,The 10th I turn 21,None
6,I am 15 years old,None
7,~20,None


Notice the use of _expand_. This allows that each element in the list goes to a column. However, as there is **no consistent pattern**, location of the symbol ":", you do not get a good result. The situation requires the **REGEX** approach. 

Using regular expressions is a great way to go when there is no pattern to apply the previous strategy; however, it takes time to learn how to build a regular expression that will serve all your especifi  needs in a particular situation.

In general, you need to **explore** few *regex pattern*s before deciding what to use. I recommend using **contains()** for that:

In [8]:
# do each cell has a character that is not a number? (\D)
someData.age.str.contains(pat=r'\D',
                          regex=True)

0    True
1    True
2    True
3    True
4    True
5    True
6    True
7    True
Name: age, dtype: bool

In [9]:
# do each cell has a number character? (\d)
someData.age.str.contains(pat=r'\d',regex=True)

0     True
1     True
2     True
3     True
4    False
5     True
6     True
7     True
Name: age, dtype: bool

In [10]:
# what is that cell?
someData[~someData.age.str.contains(pat=r'\d',regex=True)]

,age
4,I am twenty


In [11]:
# is there a cell where you have 
# symbols beyond [^ ] alphanumeric (\w) or spaces (\s)?  
someData.age[someData.age.str.contains(pat=r'[^\w\s]',regex=True)]

0     It is:24
1    It is: 44
7          ~20
Name: age, dtype: object

In [12]:
# what happens if I erase all non numbers (\D)?
someData.age.str.replace(pat=r'\D',repl='',regex=True)

0      24
1      44
2      54
3      64
4        
5    1021
6      15
7      20
Name: age, dtype: object

In [13]:
# what happens if I extract consecutive numeric characters (\d+) ?
someData.age.str.extract(pat=r'(\d+)',expand=True)

,0
0,24
1,44
2,54
3,64
4,NaN
5,10
6,15
7,20


In [14]:
# what happens if I erase all 
# numbers (\d+) followed by a texts [[a-z]+] ?
someData.age.str.replace(pat=r'\d+[a-z]+',
                         repl='',
                         regex=True)

0             It is:24
1            It is: 44
2              It is54
3             64 it is
4          I am twenty
5       The  I turn 21
6    I am 15 years old
7                  ~20
Name: age, dtype: object

In [15]:
# so:
someData.age.str.replace(pat=r'\d+[a-z]+',
                         repl='',
                         regex=True).\
             str.extract(pat=r'(\d+)',expand=True)

,0
0,24
1,44
2,54
3,64
4,NaN
5,21
6,15
7,20


In [16]:
# using or '|'
someData.age.str.extract(pat=r'(^\d+|\d+$|\s\d+\s)',
                         expand=True)

,0
0,24
1,44
2,54
3,64
4,NaN
5,21
6,15
7,20


Let me use both results:

In [17]:
someData['age1']=someData.age.str.replace(pat=r'\d+[a-z]+',
                                          repl='',
                                          regex=True).\
                                str.extract(pat=r'(\d+)',expand=True)

someData['age2']=someData.age.str.extract(pat=r'(^\d+|\d+$|\s\d+\s)',
                         expand=True)

In [18]:
someData

,age,age1,age2
0,It is:24,24,24
1,It is: 44,44,44
2,It is54,54,54
3,64 it is,64,64
4,I am twenty,NaN,NaN
5,The 10th I turn 21,21,21
6,I am 15 years old,15,15
7,~20,20,20


In [19]:
someData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   age     8 non-null      object
 1   age1    7 non-null      object
 2   age2    7 non-null      object
dtypes: object(3)
memory usage: 320.0+ bytes


In [20]:
someData['age1'].to_list()==someData['age2'].to_list()

False

In [21]:
someData['age1']==someData['age2']

0     True
1     True
2     True
3     True
4    False
5     True
6    False
7     True
dtype: bool

In [22]:
set(someData['age1'].to_list()) & set(someData['age2'].to_list())

{'20', '21', '24', '44', '54', '64', nan}

In [23]:
set(someData['age1'].to_list()) ^ set(someData['age2'].to_list())

{' 15 ', '15'}

In [24]:
someData['age1'].to_list()

['24', '44', '54', '64', nan, '21', '15', '20']

In [25]:
someData['age2'].to_list()

['24', '44', '54', '64', nan, '21', ' 15 ', '20']

In [26]:
someData['age2'].str.strip().to_list()

['24', '44', '54', '64', nan, '21', '15', '20']

In [ ]:
import os

thePath=os.path.join('DataFiles','clean_FakeDate.csv')
someData.to_csv(thePath,index=False)

## Exercise:

There is a table on Ancient wars in this wiki page:

In [1]:
from IPython.display import IFrame  
wikiLink="https://en.wikipedia.org/wiki/List_of_wars_by_death_toll" 
IFrame(wikiLink, width=900, height=500)

Download only that table and clean the column **Date**. You should:

* Keep only that column.
* Create three columns:
    - start
    - end
    - BC, a logical column indicating if the war started before AD.
    
These are the first steps:

In [3]:
import pandas as pd

#get data
ancientWars=pd.read_html(io=wikiLink,
                  flavor='bs4',
                  attrs = {'class': 'sortable'})
#check column names
ancientWars

[                                    War                Death range  \
 0                          World War II     50–85 million[4][5][6]   
 1        Mongol invasions and conquests   20–57 million[9][10][11]   
 2                   An Lushan Rebellion          13–36 million[12]   
 3                    Three Kingdoms War             34 million[13]   
 4                     Taiping Rebellion      20–30 million[14][15]   
 5                           World War I  15–30 million[16][17][18]   
 6              Manchu Conquest of China         25 million[19][20]   
 7                     Russian Civil War           7–12 million[21]   
 8                     Thirty Years' War           4–12 million[22]   
 9            Spanish conquest of Mexico       10.5 million[23][24]   
 10  Spanish conquest of the Inca Empire             10 million[25]   
 11                    Chinese Civil War            4–9 million[26]   
 12                          Reconquista              7 million[27]   
 13   

In [ ]:
# keep one column
DF_toClean=ancientWars[['Date']]
DF_toClean